# Order conditions for commutator-free exponential time propagators (CFET)

This Julia notebook requires the package `Giac`, see https://github.com/HaraldHofstaetter/Giac.jl.

In [1]:
using CFET_OrderConditions
using Giac

It holds
$$\left(\frac{d}{dt}\right)^k\mathrm{e}^{tB(t)} = \mathcal{C}_k(t)\mathrm{e}^{tB(t)},$$
where $C_k$ satisfies the recursion
$$\mathcal{C}_0(t)=\mathrm{Id},\quad\mathcal{C}_1(t) = C(t),\qquad \mathcal{C}_k(t)=\mathcal{C}_{k-1}(t)\cdot C(t)+\frac{d}{dt}\mathcal{C}_{k-1}(t)$$
with
$$C(t) = B(t)+\sum_{k=0}^{\infty}\frac{t^{k+1}}{(k+1)!}\mathrm{ad}_{B(t)}(B'(t)).$$
Here $\mathrm{ad}_{B(t)}^k(B'(t))$ denotes the iterated commutator
$$\mathrm{ad}_{B(t)}^k(B'(t))=[\underbrace{B(t),[B(t),\dots,[B(t)}_{\mbox{$k$ times}},B'(t)]\dots]].$$

The following function implements this recursion:

In [2]:
gen_exp_tBt_derivatives_coeffs(4)

5-element Array{Dict{Array{Int64,1},Int64},1}:
 Dict(Int64[]=>1)                                                                     
 Dict([0]=>1)                                                                         
 Dict([1]=>1,[0,0]=>1)                                                                
 Dict([0,1]=>1,[2]=>1,[1,0]=>2,[0,0,0]=>1)                                            
 Dict([3]=>1,[2,0]=>3,[0,0,1]=>1,[1,1]=>3,[0,1,0]=>2,[1,0,0]=>3,[0,0,0,0]=>1,[0,2]=>1)

Here, the keys Int64[], [0], [1], [0,0], [0,1], ... correspond respectively to  $\mathrm{Id}, C(t), C'(t), C(t)^2, C(t)\cdot C'(t),\dots$,
and the values are the corresponding coefficients,  
such that
(arguments of $\mathcal{C}_k(t), C(t), C'(t), \dots$ ommited)
$$\mathcal{C}_0 = \mathrm{Id},$$
$$\mathcal{C}_1 = C,$$
$$\mathcal{C}_2 = C^2+ C',$$
$$\mathcal{C}_3 = C^3 +2C'C+ CC'+C'',$$
$$\mathcal{C}_4 = C^4 +3C'C^2+C^2C'+2CC'C+3C''C+CC''+C'''.$$

It holds $C(0)=B(0)$, and for the derivatives of $C(t)$ evaluated at $t=0$ it holds
$$C^{(n)}(0) = B^{(n)}(0) +\sum_{k=0}^{n-1}{n \choose k+1}\left[\left(\frac{d}{dt}\right)^{n-k-1}\mathrm{ad}_{B(t)}^k(B'(t))\right]_{t=0},\quad n=1,2,\dots.$$
The following function computes these derivatives:

In [3]:
gen_C_derivatives_at_0(4)

5-element Array{Dict{Array{Int64,1},Int64},1}:
 Dict([0]=>1)                                                     
 Dict([1]=>2)                                                     
 Dict([0,1]=>1,[2]=>3)                                            
 Dict([3]=>4,[0,0,1]=>1,[0,2]=>3)                                 
 Dict([1,0,1]=>4,[0,3]=>6,[4]=>5,[1,2]=>6,[0,0,2]=>4,[0,0,0,1]=>1)

Here, the keys [0], [1], [0,1], [0,0,1], [0,2]... correspond respectively to iterated commutators 
$B(0), B'(0), [B(0),B'(0)], [B(0),[B(0),B'(0)]], [B(0),B''(0)],\dots$, and the values are the corresponding coefficients, such that (argument 0 ommited):
$$C=B,$$
$$C'=2B',$$
$$C''=3B''+[B,B'],$$
$$C'''=4B'''+3[B,B'']+[B,[B,B']],$$
$$C^{(4)}=5B^{(4)}+6[B',B'']+6[B,B''']+4[B',[B,B']]+4[B,[B,B'']]+[B,[B,[B,B']]].$$

We use the following parameters for our examples:

In [4]:
J, K, p = 2, 2, 4

(2,2,4)

Substitute 
$$B^{(q)}=b_qA^{(q)},\quad b_q=\sum_{l}a_lc_l^q$$
in the above equations for $C, C',C''',...$: 

In [5]:
b = giac[giac(string("b",k)) for k=0:p] 

5-element Array{Giac.giac,1}:
 b0
 b1
 b2
 b3
 b4

In [6]:
C = gen_C_derivatives_at_0_in_terms_of_A(p, b)
C = [Dict{Array{Int64,1},Giac.giac}([key=>factor(val) for (key,val) in c]) for c in C]

5-element Array{Dict{Array{Int64,1},Giac.giac},1}:
 Dict{Array{Int64,1},Giac.giac}([0]=>b0)                                                                                         
 Dict{Array{Int64,1},Giac.giac}([1]=>2*b1)                                                                                       
 Dict{Array{Int64,1},Giac.giac}([0,1]=>b1*b0,[2]=>3*b2)                                                                          
 Dict{Array{Int64,1},Giac.giac}([3]=>4*b3,[0,0,1]=>b0^2*b1,[0,2]=>3*b2*b0)                                                       
 Dict{Array{Int64,1},Giac.giac}([1,0,1]=>4*b1^2*b0,[0,3]=>6*b3*b0,[4]=>5*b4,[1,2]=>6*b2*b1,[0,0,2]=>4*b0^2*b2,[0,0,0,1]=>b0^3*b1)

Now the keys [0], [1], [0,1], [0,0,1], [0,2]... correspond respectively to iterated commutators 
$A(0), A'(0), [A(0),A'(0)], [A(0),[A(0),A'(0)]], [A(0),A''(0)],\dots$, and the values are the corresponding coefficients (which are Giac expressions), such that (argument 0 ommited):
$$C = b_0A,$$
$$C' = 2b_1 A',$$
$$C'' = 3b_2 A''+b_0b_1[A,A'],$$
$$C''' = 4b_3A''' +3b_0b_2[A,A'']+b_0^2b_1[A,[A,A']],$$
$$C^{(4)}=5b_4A^{(4)}+6b_1b_2[A',A'']+6b_0b_3[A,A''']+4b_1^2b_0[A',[A,A']]+4b_0^2b_2[A,[A,A'']]+b_0^3b_1[A,[A,[A,A']]].$$

Now we can compute
$$\left[\left(\frac{d}{dt}\right)^k\mathrm{e}^{tB(t)}\right]_{t=0} = \big[\mathcal{C}_{k}(t)\mathrm{e}^{tB(t)}\big]_{t=0}=\mathcal{C}_{k}(0),\quad k=1,2,\dots$$
with the $\mathcal{C}_k$ form above. For $k=0$ it holds $\left[\mathrm{e}^{tB(t)}\right]_{t=0}=\mathrm{Id}$ of course.

In [7]:
C = gen_exp_tBt_derivatives_at_0_in_terms_of_A(p, b)
C = [LCCC{Giac.giac}([key=>factor(val) for (key,val) in c]) for c in C]

5-element Array{Dict{Array{Array{Int64,1},1},Giac.giac},1}:
 Dict{Array{Array{Int64,1},1},Giac.giac}([Int64[]]=>1)                                                                                                                                                                                                                                           
 Dict{Array{Array{Int64,1},1},Giac.giac}([[0]]=>b0)                                                                                                                                                                                                                                              
 Dict{Array{Array{Int64,1},1},Giac.giac}([[1]]=>2*b1,[[0],[0]]=>b0^2)                                                                                                                                                                                                                            
 Dict{Array{Array{Int64,1},1},Giac.giac}([[0,1]]=>b1*b0,[[2]]=>3*b2,[[

The output is to be interpreted as follows (argument 0 ommited):
$$\mathcal{C}_{0} = \mathrm{Id},$$
$$\mathcal{C}_{1} = b_0A,$$
$$\mathcal{C}_{2} = 2b_1A'+ b_0^2A^2,$$
$$\mathcal{C}_{3} = 3b_2A''+b_0b_1[A,A']+4b_0b_1A'A+2b_0b_1AA'+b_0^3A^3$$
$$\mathcal{C}_{4} = 4b_3A'''+3b_0b_2AA''+9b_0b_2A''A+3b_0b_2[A,A'']+4b_0^2b_1AA'A+6b_0^2b_1A'A^2+2b_0^2b_1A^2A'+12b_1^2(A')^2
+b_0^2b_1A[A,A']+3b_0^2b_1[A,A']A+b_0^2b_1[A,[A,A']]+b_0^4A^4.$$
Here it seems reasonable to expand all commutators, because the expanded terms are present in the expressions anyway.

In [8]:
C = [expand_commutators(c) for c in C]
C = [LCCC{Giac.giac}([key=>factor(val) for (key,val) in c]) for c in C]

5-element Array{Dict{Array{Array{Int64,1},1},Giac.giac},1}:
 Dict{Array{Array{Int64,1},1},Giac.giac}([Int64[]]=>1)                                                                                                                                                           
 Dict{Array{Array{Int64,1},1},Giac.giac}([[0]]=>b0)                                                                                                                                                              
 Dict{Array{Array{Int64,1},1},Giac.giac}([[1]]=>2*b1,[[0],[0]]=>b0^2)                                                                                                                                            
 Dict{Array{Array{Int64,1},1},Giac.giac}([[1],[0]]=>3*b1*b0,[[2]]=>3*b2,[[0],[0],[0]]=>b0^3,[[0],[1]]=>3*b0*b1)                                                                                                  
 Dict{Array{Array{Int64,1},1},Giac.giac}([[0],[0],[0],[0]]=>b0^4,[[1],[0],[0]]=>4*b0^2*b1,[[0],[0],[

The output is to be interpreted as follows (argument 0 ommited):
$$\mathcal{C}_{0} = \mathrm{Id},$$
$$\mathcal{C}_{1} = b_0A,$$
$$\mathcal{C}_{2} = 2b_1A'+ b_0^2A^2,$$
$$\mathcal{C}_{3} = 3b_2A''+3b_0b_1A'A+3b_0b_1AA'+b_0^3A^3,$$
$$\mathcal{C}_{4} = 4b_3A'''+6b_0b_2A''A+6b_0b_2AA''+12b_1^2(A')^2+4b_0^2b_1A'A^2+4b_0^2b_1AA'A+4b_0^2b_1A^2A'+b_0^4A^4$$

Numerical method:
$$S(t) = \prod_{j=J,\dots,1}\mathrm{e}^{tB_j(t)}.$$
Defect:
$$D(t) = \frac{d}{dt}S(t)-A(t)S(t).$$
Derivatives of the defect are calculated using the Leibniz formula for higher derivatives:
$$D^{(q)}(t) = \sum_{\mathbf{k}=(k_1,\dots,k_J)\atop |\mathbf{k}|=q+1}{q+1 \choose \mathbf{k}}\prod_{j=J,\dots,1}\left(\frac{d}{dt}\right)^{k_j}\mathrm{e}^{tB_j(t)}-
\sum_{\mathbf{k}=(k_1,\dots,k_{J+1})\atop |\mathbf{k}|=q}{q \choose \mathbf{k}}A^{(k_{J+1})}(t)\prod_{j=J,\dots,1}\left(\frac{d}{dt}\right)^{k_j}\mathrm{e}^{tB_j(t)}.
$$

Substituting $\left(\frac{d}{dt}\right)^k\mathrm{e}^{tB_j(t)} = \mathcal{C}_{j,k}(t)\mathrm{e}^{tB_j(t)}$ followed
by an evaluation at $t=0$ yields
$$D^{(q)}(0) = \sum_{\mathbf{k}=(k_1,\dots,k_J)\atop |\mathbf{k}|=q+1}{q+1 \choose \mathbf{k}}\prod_{j=J,\dots,1}\mathcal{C}_{j,k_j}(0)-\sum_{\mathbf{k}=(k_1,\dots,k_{J+1})\atop |\mathbf{k}|=q}{q \choose \mathbf{k}}A^{(k_{J+1})}(0)\prod_{j=J,\dots,1}\mathcal{C}_{j,k_j}(0).$$

The following function computes $D^{(q)}(0)$ for $q=0,\dots,p-1$. To obtain order $p$ for the method $S(t)$, all
coefficients of $D^{(q)}(0), q=0,\dots,p-1$ are required to vanish => order conditions.

In [9]:
D = gen_CFET_order_conditions(p,J)

4-element Array{Dict{Array{Int64,1},Giac.giac},1}:
 Dict{Array{Int64,1},Giac.giac}([0]=>b2_0+b1_0-1)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               
 Dict{Array{Int64,1},Giac.giac}([1]=>2*b2_1+2*b1_1-1,[0,0]=>(b2_0+b1_0)*(b2_0+b1_0-1))                                                                                                                                                                                                                                                                                                              

Conjecture: If the order conditions are satisfied up to order $p-1$ then the leading term of the local error of order $p$ is
an element of grade $p$ of the graded Lie-algebra with generators $\{A,A'',A''',\dots\}$
with grades
$$w(A^{(n)})=n+1,\quad n =0,1,2,\dots.$$
From this conjecture it would follow that many of the computed order conditions are redundant. We only keep 
those, whose indices correspond to Hall basis elements of grade <= the required convergence order p. 
For $p\leq 10$ these are the following basis elements. (`gen_hall_basis_elements_of_grade`  is based on http://mathoverflow.net/questions/97703/list-of-hall-basis.)

In [10]:
for q=1:10
    println("grade ", q, ":\n",gen_hall_basis_elements_of_grade(q))
end

grade 1:
[[0]]
grade 2:
[[1]]
grade 3:
[[2],[0,1]]
grade 4:
[[3],[0,2],[0,0,1]]
grade 5:
[[4],[0,3],[1,2],[0,0,2],[0,1,1],[0,0,0,1]]
grade 6:
[[5],[0,4],[1,3],[0,0,3],[0,1,2],[0,2,1],[0,0,0,2],[0,0,1,1],[0,0,0,0,1]]
grade 7:
[[6],[0,5],[1,4],[0,0,4],[2,3],[0,1,3],[0,3,1],[0,0,0,3],[0,2,2],[1,1,2],[0,0,1,2],[0,0,2,1],[0,1,0,2],[0,0,0,0,2],[0,1,1,1],[0,0,0,1,1],[0,0,1,0,1],[0,0,0,0,0,1]]
grade 8:
[[7],[0,6],[1,5],[0,0,5],[2,4],[0,1,4],[0,4,1],[0,0,0,4],[0,2,3],[0,3,2],[1,1,3],[0,0,1,3],[0,0,3,1],[0,1,0,3],[0,0,0,0,3],[1,2,2],[0,0,2,2],[0,1,1,2],[0,1,2,1],[0,2,1,1],[0,0,0,1,2],[0,0,0,2,1],[0,0,1,0,2],[0,0,2,0,1],[0,0,0,0,0,2],[0,0,1,1,1],[0,1,0,1,1],[0,0,0,0,1,1],[0,0,0,1,0,1],[0,0,0,0,0,0,1]]
grade 9:
[[8],[0,7],[1,6],[0,0,6],[2,5],[0,1,5],[0,5,1],[0,0,0,5],[3,4],[0,2,4],[0,4,2],[1,1,4],[0,0,1,4],[0,0,4,1],[0,1,0,4],[0,0,0,0,4],[0,3,3],[1,2,3],[1,3,2],[0,0,2,3],[0,0,3,2],[0,2,0,3],[0,1,1,3],[0,1,3,1],[0,3,1,1],[0,0,0,1,3],[0,0,0,3,1],[0,0,1,0,3],[0,0,3,0,1],[0,0,0,0,0,3],[0,1,2,2],[0,2,1

The number of basis elements of grade $n$ is given by the formula
$$\nu_n=\frac{1}{n}\sum_{d|n}2^{n/d}\mu(d).$$
The function `number_commutators_of_grade_equal_to` computes $\nu_n$ according to this formula:

In [11]:
s = 0
println("n\tnu\tsum")
println("--------------------")
for n=1:10
    t = number_commutators_of_grade_equal_to(n)
    s+=t
    println(n,"\t",t,"\t",s)
end

n	nu	sum
--------------------
1	1	1
2	1	2
3	2	4
4	3	7
5	6	13
6	9	22
7	18	40
8	30	70
9	56	126
10	99	225


The sequence $\{\nu_n\}$ is  [A059966](http://oeis.org/A059966) in the [The On-Line Encyclopedia of Integer Sequences](http://oeis.org). Note that for n>=2 the number of order conditions is exactly the same as for splitting methods.

The following function computes only those order conditions, which correspond to Hall basis elements:

In [12]:
D = gen_CFET_order_conditions_without_redundancies(p,J)

4-element Array{Dict{Array{Int64,1},Giac.giac},1}:
 Dict{Array{Int64,1},Giac.giac}([0]=>b2_0+b1_0-1)                                                                                                                                                                       
 Dict{Array{Int64,1},Giac.giac}([1]=>2*b2_1+2*b1_1-1)                                                                                                                                                                   
 Dict{Array{Int64,1},Giac.giac}([0,1]=>6*b2_0*b1_1+3*b2_0*b2_1+3*b1_1*b1_0-2*b1_1-2*b2_1,[2]=>3*b2_2+3*b1_2-1)                                                                                                          
 Dict{Array{Int64,1},Giac.giac}([3]=>4*b2_3+4*b1_3-1,[0,0,1]=>12*b1_1*b2_0^2+12*b1_1*b2_0*b1_0-6*b1_1*b2_0+4*b1_1*b1_0^2-3*b1_1*b1_0+4*b2_0^2*b2_1-3*b2_0*b2_1,[0,2]=>3*(4*b2_0*b1_2+2*b2_0*b2_2+2*b1_2*b1_0-b1_2-b2_2))

As an example we consider the following CFET of order 4:
  $$(c_1, c_2)= \left(\frac{1}{2}-\frac{\sqrt{3}}{6}, \frac{1}{2}+\frac{\sqrt{3}}{6}\right),\quad
  (a_{11}, a_{12}) = \left(\frac{1}{4}+\frac{\sqrt{3}}{6}, \frac{1}{4}-\frac{\sqrt{3}}{6}\right),\quad
  (a_{21}, a_{22}) = \left(\frac{1}{4}-\frac{\sqrt{3}}{6}, \frac{1}{4}+\frac{\sqrt{3}}{6}\right)$$.

In [13]:
c = [1//2-sqrt(giac(3))/6, 1//2+sqrt(giac(3))/6]

2-element Array{Giac.giac_SYMB,1}:
 1/2-sqrt(3)/6
 1/2+sqrt(3)/6

In [14]:
a = Array[[1//4+sqrt(giac(3))/6, 1//4-sqrt(giac(3))/6],
          [1//4-sqrt(giac(3))/6, 1//4+sqrt(giac(3))/6]]

2-element Array{Array{T,N},1}:
 [1/4+sqrt(3)/6,1/4-sqrt(3)/6]
 [1/4-sqrt(3)/6,1/4+sqrt(3)/6]

The corresponding coefficients $b_{jk}=\sum_l a_{jl}c_l^k$ are nice rational numbers:

In [15]:
b = [[simplify(sum([a[j][l]*c[l]^k for l=1:K])) for k=0:p-1] for j=1:J]

2-element Array{Array{T,N},1}:
 Any[1/2,1/12,0,-1/72]  
 Any[1/2,5/12,1/3,19/72]

In [16]:
bb = [[giac(string("b",j,"_",n)) for n=0:p-1] for  j=1:J]

2-element Array{Array{T,N},1}:
 Any[b1_0,b1_1,b1_2,b1_3]
 Any[b2_0,b2_1,b2_2,b2_3]

We substitute these values $b_{jk}$ into the expressions for $D^{(q)}, q=0\dots, p-1$:

In [17]:
[Dict{Array{Int,1},Int}([key=>simplify(subst(subst(val, bb[1], b[1]), bb[2], b[2])) for (key,val) in D[q+1]]) for q=0:p-1]

4-element Array{Dict{Array{Int64,1},Int64},1}:
 Dict([0]=>0)                    
 Dict([1]=>0)                    
 Dict([0,1]=>0,[2]=>0)           
 Dict([3]=>0,[0,0,1]=>0,[0,2]=>0)

All coefficients are zero as required.

For the construction of a 'local error measure' (LEM) we compute $D^{(p)}(0)$:

In [18]:
Dnext = gen_CFET_order_conditions_without_redundancies(p+1,J);
bb = [[giac(string("b",j,"_",n)) for n=0:p] for  j=1:J]
vars = vcat(bb...)
b = [[simplify(sum([a[j][l]*c[l]^k for l=1:K])) for k=0:p] for j=1:J]
b = vcat(b[1], b[2])
[Dict{Array{Int,1},giac}([key=>simplify(subst(val, vars, b)) for (key,val) in Dnext[q+1]]) for q=0:p]

5-element Array{Dict{Array{Int64,1},Giac.giac},1}:
 Dict{Array{Int64,1},Giac.giac}([0]=>0)                                                                      
 Dict{Array{Int64,1},Giac.giac}([1]=>0)                                                                      
 Dict{Array{Int64,1},Giac.giac}([0,1]=>0,[2]=>0)                                                             
 Dict{Array{Int64,1},Giac.giac}([3]=>0,[0,0,1]=>0,[0,2]=>0)                                                  
 Dict{Array{Int64,1},Giac.giac}([0,3]=>1/9,[0,1,1]=>1/18,[4]=>-1/36,[1,2]=>1/6,[0,0,2]=>-1/6,[0,0,0,1]=>1/24)

For the given method we compute the LEM:

In [19]:
norm([1/9,1/18,-1/36,1/6,-1/6,1/24])

0.27110029577698797

Now we try to find all solutions for $J=2$, $p=4$.

In [20]:
eqs=vcat([collect(values(D[q])) for q=1:p]...)

7-element Array{Giac.giac,1}:
 b2_0+b1_0-1                                                                                     
 2*b2_1+2*b1_1-1                                                                                 
 6*b2_0*b1_1+3*b2_0*b2_1+3*b1_1*b1_0-2*b1_1-2*b2_1                                               
 3*b2_2+3*b1_2-1                                                                                 
 4*b2_3+4*b1_3-1                                                                                 
 12*b1_1*b2_0^2+12*b1_1*b2_0*b1_0-6*b1_1*b2_0+4*b1_1*b1_0^2-3*b1_1*b1_0+4*b2_0^2*b2_1-3*b2_0*b2_1
 3*(4*b2_0*b1_2+2*b2_0*b2_2+2*b1_2*b1_0-b1_2-b2_2)                                               

In [21]:
bb = [[giac(string("b",j,"_",n)) for n=0:p-1] for  j=1:J]
vars = vcat(bb...)

8-element Array{Any,1}:
 b1_0
 b1_1
 b1_2
 b1_3
 b2_0
 b2_1
 b2_2
 b2_3

`Giac/solve` finds all solutions, they have one free parameter $b_{2,3}$ (for the CFET from above $b_{2,3}=\frac{19}{72}$):


In [22]:
sols = to_julia(solve(eqs, vars))[1] 

8-element Array{Any,1}:
 1//2       
 1//12      
  0         
   -b2_3+1/4
 1//2       
 5//12      
 1//3       
   b2_3     

For the construction of a 'local error measure' (LEM) we compute $D^{(p)}(0)$:

In [23]:
Dnext = gen_CFET_order_conditions_without_redundancies(p+1,J);
Dict{Array{Int,1},giac}([key=>simplify(subst(val, vars, sols)) for (key,val) in Dnext[p+1]])

Dict{Array{Int64,1},Giac.giac} with 6 entries:
  [0,3]     => (-40*b2_3+11)/4
  [0,1,1]   => 1/18
  [4]       => 5*b1_4+5*b2_4-1
  [1,2]     => 1/6
  [0,0,2]   => -1/6
  [0,0,0,1] => 1/24

If we could manage to get zero for the not yet determined elements `(-40*b2_3+11)/4` and `5*b1_4+5*b2_4-1`, the
corresponding LEM would be:

In [24]:
norm([0,1/18,0,1/6,-1/6,1/24])

0.24571952795769628

This is not much smaller than the LEM=0.27110029577698797 of the method from above. Therefore, it seems that  further optimization for the case $J=K=2, p=4$ is not worth the trouble.